In [15]:
import pandas as pd
import sqlite3

## create a connection to the database using the library sqlite3

In [16]:
db = sqlite3.connect('../data/checking-logs.sqlite.sqlite_copy')

## create a new table datamart in the database by joining the tables pageviews and checker using only one query the table should have the

In [17]:
query = '''
SELECT uid, labname, min(timestamp) as first_commit_ts, first_view_ts FROM checker
LEFT JOIN
    (SELECT uid as uid_pv, min(datetime) as first_view_ts FROM pageviews
    WHERE uid LIKE "user_%"
    GROUP BY uid_pv) as pv ON checker.uid = pv.uid_pv
WHERE status = "ready"
AND numTrials = 1
AND uid LIKE "user_%"
AND labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s", "project1")
GROUP BY labname, uid ;'''
pd.io.sql.read_sql(query, db, parse_dates=['first_commit_ts', 'first_view_ts']).to_sql('datamart', db, if_exists='replace', index=False)

## using Pandas methods, create two dataframes: test and control

In [18]:
query = 'SELECT * FROM datamart ;'
df = pd.io.sql.read_sql(query, db, parse_dates=['first_commit_ts', 'first_view_ts'])

## test should have the users that have the values in first_view_ts. Control should have the users that have missing values in first_view_ts

In [19]:
with_views = df['first_view_ts'].notnull()
no_views = df['first_view_ts'].isnull()
test = df[with_views].copy()
control = df[no_views].copy()

## replace the missing values in the control with the average first_view_ts of the test users, we will use this value for the future analysis

In [20]:
control.fillna({'first_view_ts' : test['first_view_ts'].mean()}, inplace=True)
test.to_sql('test', db, if_exists='replace', index=False)
control.to_sql('control', db, if_exists='replace', index=False)

## close the connection

In [21]:
db.close()